# Model training and prediction - `s30d`

In [1]:
import os
os.chdir("..")

In [2]:
os.getcwd()

'/Users/ludvigwarnberggerdin/projects/ttris/pemett'

In [3]:
import numpy as np
import pandas as pd

from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier, RandomForestClassifier

In [4]:
X_train = pd.read_csv("./data/processed/s30d/X_train.csv", index_col = 0)
y_train = pd.read_csv("./data/processed/s30d/y_train.csv", index_col = 0).s30d
X_test = pd.read_csv("./data/processed/s30d/X_test.csv", index_col = 0)
y_test = pd.read_csv("./data/processed/s30d/y_test.csv", index_col = 0).s30d

In [5]:
y_train.value_counts() / len(y_train.index) * 100

0.0    94.197074
1.0     5.802926
Name: s30d, dtype: float64

In [6]:
cont_features = ["age", "hr", "sbp", "dbp", "spo2", "rr", "delay"]
cat_features = list(X_train.loc[:, ~X_train.columns.isin(cont_features)].columns)

### Lightgbm

In [7]:
from lightgbm import LGBMClassifier

In [8]:
continous_transformer = StandardScaler()
preprocessor = ColumnTransformer(
    transformers=[
        ('cont', continous_transformer, cont_features)]
)

In [9]:
ss = StandardScaler()
X_train.loc[:, cont_features] = ss.fit_transform(X_train.loc[:, cont_features])
X_test.loc[:, cont_features] = ss.fit_transform(X_test.loc[:, cont_features])

In [10]:
clf = LGBMClassifier()

In [11]:
clf.fit(
    X = X_train,
    y = y_train,
    categorical_feature = cat_features
)

/Users/ludvigwarnberggerdin/miniforge3/envs/pemett/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


LGBMClassifier()

In [12]:
y_pred_prob_train = clf.predict_proba(X = X_train)
y_pred_prob_test = clf.predict_proba(X = X_test)
y_pred_train = clf.predict(X = X_train)
y_pred_test = clf.predict(X = X_test)

Report for continous scores

In [13]:
print(classification_report(y_true = y_train, y_pred = y_pred_train))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      5860
         1.0       1.00      0.99      1.00       361

    accuracy                           1.00      6221
   macro avg       1.00      1.00      1.00      6221
weighted avg       1.00      1.00      1.00      6221



In [14]:
print(classification_report(y_true = y_test, y_pred = y_pred_test))

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98      1954
         1.0       0.77      0.62      0.69       120

    accuracy                           0.97      2074
   macro avg       0.87      0.80      0.83      2074
weighted avg       0.96      0.97      0.97      2074



Gridsearch breaks for the continous score (to enable comparison with clinicians)

In [15]:
import copy
import random
from src.models.train_model import generate_all_combinations
from tqdm.notebook import tqdm

from typing import Callable, Optional

from sklearn.model_selection import cross_val_predict, StratifiedKFold

In [100]:
def cv_inner_loop(base_clfs: list, hyper_parameters: dict,
                  inner_loop: Callable, 
                  X: pd.DataFrame, 
                  y: pd.Series) -> np.ndarray:
    """Run inner loop of k-fold cross-validation.
    
    Uses sklearn's cross_val_predict.
    
    That is,
    1. Fit classifier to the training folds.
    2. Make prediction on the validation fold.
    3. Use all folds as validation fold, one time each.

    Args:
      base_clfs: List of classifiers. E.g. [LGBMClassifier, LogisticRegression]
      inner_loop: scikit-learn callable to split into folds
      X: Features
      y: Targets

    Returns:
      Each column represent predictions by each respective classifier
    """
    predictions = np.zeros((len(X_train.index), ))
    for clfk in base_clfs.keys(): 
        ks = [s for s in hyper_parameters.keys() if clfk in s]
        clf_params = {k.split("__")[1]: hyper_parameters.get(k) for k in ks}
        clf = base_clfs[clfk]
        clf.set_params(**clf_params)
        print("Running predictions for " + str(clf) + " with params " +  str(clf_params))
        preds = cross_val_predict(
            estimator=clf,
            X=X,
            y=y,
            cv=inner_loop
        )
        if predictions.any():
            predictions = np.hstack([predictions, preds[:, np.newaxis]])
        else:
            predictions = preds[:, np.newaxis]

    return predictions

In [162]:
def cv_outer_loop(base_clfs: list, meta_clf: Callable,
                  all_hyper_parameters: list,
                  X: pd.DataFrame, y: pd.Series,
                  use_meta_features: bool = False, 
                  **kwargs):
    """Conduct outer cross-validation.
    
    That is, find the best combination cut-points for the classifier.
    
    Inspired by:
        https://github.com/rasbt/mlxtend/blob/master/mlxtend/classifier/stacking_cv_classification.py
        
    Args:
        base_clfs: Base classifiers.
        meta_clf:
        X_train:
        y_train
        use_meta_features
    """
    ## Setup splitting
    inner_folds = 5
    outer_folds = 2
    inner_loop = StratifiedKFold(n_splits = inner_folds)
    outer_loop = StratifiedKFold(n_splits = outer_folds)
    
    ## Setup for recording auc from each combination of hps
    roc_aucs = pd.DataFrame(
        data = np.zeros((len(all_hyper_parameters), outer_folds)),
        columns = range(1, outer_folds + 1),
        index = [str(hp) for hp in all_hyper_parameters]
    )
    
    for i, hyper_parameters in enumerate(all_hyper_parameters):
        
        breaks = hyper_parameters.pop("breaks")

        for j, (train_index, val_index) in enumerate(outer_loop.split(X, y)):
        
            X_train = X.iloc[train_index]
            y_train = y.iloc[train_index]
            X_val = X.iloc[val_index]
            y_val = y.iloc[val_index]
        
            inner_predictions = cv_inner_loop(
                base_clfs=base_clfs,
                hyper_parameters=hyper_parameters,
                inner_loop=inner_loop,
                X=X_train,
                y=y_train
            )
            
            if use_meta_features: pass
            
            # Fit meta classifier to base predictions
            meta_clf.fit(inner_predictions, y_train)
            print(X_val)
            y_pred_prob_meta = meta_clf.predict_proba(X_val)
            binned_predictions = pd.cut(
                x=y_pred_prob_meta[:, 1],
                bins=breaks,
                labels=[0, 1, 2, 3],
                right=True,
                include_lowest=False
            )
            roc_aucs.iloc[j, i] = roc_auc_score(
                y_true=y_val,
                y_score=binned_predictions
            )

    return base_clfs, meta_clf

In [163]:
import itertools as it
all_breaks = [(0, ) + x + (np.inf,) for x in it.combinations(np.arange(0.01, 1, 0.01), r=3)]

In [164]:
hyper_parameters = {
    "lgbm1__max_depth": [100, 500],
    "lgbm1__num_leaves": [200, 100],
    "lgbm2__max_depth": [100, 200],
    "breaks": all_breaks[:5]
}

In [165]:
base_clfs = {
    "lgbm1": LGBMClassifier(),
    "lgbm2": LGBMClassifier()
}

In [166]:
all_hyper_parameters = generate_all_combinations(hyper_parameters)

In [167]:
a, b = cv_outer_loop(
    base_clfs=base_clfs,
    meta_clf=LogisticRegression(),
    all_hyper_parameters=all_hyper_parameters,
    all_breaks=all_breaks,
    X=X_train,
    y=y_train,
)

{'lgbm1__max_depth': 100, 'lgbm1__num_leaves': 200, 'lgbm2__max_depth': 100, 'breaks': (0, 0.01, 0.02, 0.03, inf)}
Running predictions for LGBMClassifier(max_depth=100, num_leaves=200) with params {'max_depth': 100, 'num_leaves': 200}
Running predictions for LGBMClassifier(max_depth=100) with params {'max_depth': 100}
            age  moi  sex  mot  tran  egcs  mgcs  vgcs  avpu        hr  \
259   -0.665919    0    1    2     0     3     5     4     3  0.034803   
11083  0.068646    0    1    2     0     3     5     4     3  0.351697   
2865   0.068646    0    0    0     1     3     5     4     3  1.682654   
4378  -1.133369    7    1    0     1     3     5     4     3 -0.408849   
5060  -0.732697    0    0    2     0     3     5     4     3  1.999548   
...         ...  ...  ...  ...   ...   ...   ...   ...   ...       ...   
3644   0.402540    3    0    2     0     3     5     4     3 -0.282092   
7604  -1.200148    5    1    2     0     3     5     4     3 -0.472228   
3785   0.06864

/Users/ludvigwarnberggerdin/miniforge3/envs/pemett/lib/python3.10/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [78]:
clfs, meta_clf = ensemble_learning(
    base_clfs=clfs,
    meta_clf=LGBMClassifier(),
    all_breaks=all_breaks,
    X_train=X_train, 
    y_pred_prob_train=y_pred_prob_train,
    y_train = y_train,
    sample_size=0.0001
)

Running predictions for LGBMClassifier()


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

TypeError: tuple indices must be integers or slices, not tuple

In [16]:
def cv_outer_loop(base_clfs: list, meta_clf: Callable,
                  X_train: pd.DataFrame, y_train: pd.Series,
                  use_meta_features: bool = False, 
                  **kwargs):
    """Conduct outer cross-validation.
    
    That is, find the best combination cut-points for the classifier.
    
    Inspired by:
        https://github.com/rasbt/mlxtend/blob/master/mlxtend/classifier/stacking_cv_classification.py
        
    Args:
        base_clfs: Base classifiers.
        meta_clf:
        X_train:
        y_train
        use_meta_features
    """
    ## Fit each classifier to the original training set
    meta_predictions = np.array([])
    clf_keys = base_clfs.keys()
    ## Setup splitting
    inner_folds = 5
    outer_folds = 2
    inner_loop = StratifiedKFold(n_splits = inner_folds)
    outer_loop = StratifiedKFold(n_splits = outer_folds)
    
    ## Setup for recording auc from each combination of hps
    roc_aucs = pd.DataFrame(
        data = np.zeros((len(hyper_parameters), n_folds)),
        columns = range(1, n_folds + 1),
        index = [str(hp) for hp in hyper_parameters]
    )

    for i, (train_index, val_index) in enumerate(outer_loop.split(X_train, y_train)):
        
        X_train_ = X_train.iloc[train_index]
        y_train_ = y_train.iloc[train_index]

        X_val = X_train.iloc[val_index]
        y_val = y_train.iloc[val_index]
        
        inner_predictions = cv_inner_loop(
            base_clfs = base_clfs,
            
        )
        
        for j, hp in enumerate(hyper_parameters):
            
            hp = 
            breaks = hp['breaks']
            
            ## Fit each classifier to the predicted probabilities
            base_predictions = inner_loop(
                base_clfs=base_clfs
                hyper_parameters=hyper_parameters,
                inner_loop=inner_loop,
                X=X_train_,
                y=y_train_
            )
    
            ## Fit the meta clf to predictions, and predict on validation set
            meta_clf.fit(predictions, y_train)
            y_pred_prob_meta = meta_clf.predict_proba(predictions)
            binned_predictions = pd.cut(y_pred_prob_val[:, 1], breaks, labels = [0, 1, 2, 3], right = True, include_lowest = False)
            roc_aucs.iloc[j, i] = roc_auc_score(y_true = y_val, y_score = binned_predictions)
    
    return base_clfs, meta_clf

SyntaxError: invalid syntax (2054303279.py, line 45)